In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import utils
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.regularizers import l2

In [3]:
train_data_path = '/home/ec2-user/code/left-pca16' # Full path should be given here 
train_mask_path = 'mask/train/'
test_data_path = '/home/ec2-user/code/right-pca16' # Full path should be given here 
test_mask_path = 'mask/test/'

## Visualize Masks

In [ ]:
files = []
folder = 'mask/train/'
path = []
for filename in os.listdir('mask/train/'):
    files.append(filename)
    path.append('{}{}'.format(folder,filename))

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(12, 8))
axs = axs.ravel()
for i in range(len(path)):
    img = load_img(path=path[i],grayscale=True, target_size=(150,150), interpolation='nearest')
    axs[i].imshow(img)
    axs[i].set_title(files[i])

In [ ]:
files = []
folder = 'mask/test/'
path = []
for filename in os.listdir('mask/test/'):
    files.append(filename)
    path.append('{}{}'.format(folder,filename))

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(12, 8))
axs = axs.ravel()
for i in range(len(path)):
    img = load_img(path=path[i],grayscale=True, target_size=(150,150), interpolation='nearest')
    axs[i].imshow(img)
    axs[i].set_title(files[i])

In [ ]:
# Read data form training and test 

In [4]:
X_train,Y_train, num_bands = utils.load_data(data_path = train_data_path, 
                                             masks_path = train_mask_path, 
                                             crop_size = 33, 
                                             num_classes = 6, 
                                             samples = 1000, 
                                             balance = True)

In [6]:
X_test,Y_test, num_bands = utils.load_data(data_path = test_data_path, 
                                           masks_path = test_mask_path, 
                                           crop_size = 33, 
                                           num_classes = 6, 
                                           samples = 1000, 
                                           balance = True)

In [ ]:
# creat generato

In [7]:
print(type(X_train), type(Y_train))
print(X_train.shape, Y_train.shape, num_bands)
print(X_test.shape,  Y_test.shape,  num_bands)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(6000, 33, 33, 16) (6000, 6) 16
(6000, 33, 33, 16) (6000, 6) 16


In [8]:
X_train_list = np.array([X_train[i] for i in range(X_train.shape[0])])

In [9]:
X_test_list = np.array([X_test[i] for i in range(X_test.shape[0])])

In [10]:
Y_train_list = np.array([Y_train[i,:] for i in range(Y_train.shape[0])])

In [11]:
Y_test_list = np.array([Y_test[i,:] for i in range(Y_test.shape[0])])

# Build First Model

In [12]:
# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=(33, 33, 16)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(6))
# model.add(Activation('softmax'))

model = Sequential()
model.add(Conv2D(filters=8, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=(33, 33, 16),
                 kernel_regularizer=l2(0.01)))

model.add(MaxPooling2D(pool_size=(2, 2),
                       strides=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(5, 5),
                 strides=(1, 1),
                 activation='relu',
                 kernel_regularizer=l2(0.01)))

model.add(MaxPooling2D(pool_size=(2, 2),
                       strides=(2, 2)))
model.add(Flatten())
model.add(Dense(128,
                activation='relu',
                kernel_regularizer=l2(0.01)))
model.add(Dense(6,
                activation='softmax',
                kernel_regularizer=l2(0.01)))

In [13]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

In [16]:
model.fit(X_train_list, Y_train_list,
          epochs = 20,
          verbose = 1,
          batch_size= 50,
          validation_data=(X_test_list, Y_test_list))

Train on 6000 samples, validate on 6000 samples
Epoch 1/20
6000/6000 [==============================] - 2s 278us/step - loss: 0.4326 - acc: 0.9252 - val_loss: 1.2143 - val_acc: 0.6880
Epoch 2/20
6000/6000 [==============================] - 2s 279us/step - loss: 0.3934 - acc: 0.9388 - val_loss: 1.1361 - val_acc: 0.6277
Epoch 3/20
6000/6000 [==============================] - 2s 281us/step - loss: 0.3803 - acc: 0.9372 - val_loss: 1.1161 - val_acc: 0.7028
Epoch 4/20
6000/6000 [==============================] - 2s 280us/step - loss: 0.3728 - acc: 0.9400 - val_loss: 1.3523 - val_acc: 0.6535
Epoch 5/20
6000/6000 [==============================] - 2s 281us/step - loss: 0.3778 - acc: 0.9318 - val_loss: 1.2231 - val_acc: 0.6743
Epoch 6/20
6000/6000 [==============================] - 2s 278us/step - loss: 0.3678 - acc: 0.9372 - val_loss: 1.2819 - val_acc: 0.6568
Epoch 7/20
6000/6000 [==============================] - 2s 279us/step - loss: 0.3576 - acc: 0.9372 - val_loss: 0.9631 - val_acc: 0.7233
